In [1]:
import requests,re
import pandas as pd
import time


def getURL(pageNum):
    url = 'https://careers.huntington.com/en-US/search?pagenumber='
    urls = []
    for i in range(pageNum):
        urls.append(url + str(i+1))
    return urls

def getHTMLText(url):
    try:
        kv = {'user-agent': 'Mozilla/5.0'}
        r = requests.get(url, headers=kv)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        print('404 error!')
        return(' ')

def fillPositionList(html):
    time.sleep(2)
    pattern = re.compile('"url": "(.*?)"')
    p = re.findall(pattern, html)
    if p:
        p_de = sorted(set(p),key=p.index)
        return p
    return 'N'

def printCsv(sinfo):
    df = pd.DataFrame(sinfo)
    df.to_csv('position.csv', sep=',', header=True, index=True) # set output path
    #print(df)
    
def main():
    sinfo = []
    urls = getURL(6) # set the number of page you want
    i = 1
    for url in urls:
        html = getHTMLText(url)
        sinfo.extend(fillPositionList(html))
        if i % 5 == 0:
            print("%d / %d" %(i,len(urls)))
        i += 1
    printCsv(sinfo)
    
    
main()


5 / 6


In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import json
from pandas.io.json import json_normalize
from requests import get
from bs4 import BeautifulSoup as bs

In [3]:
url_pd = pd.read_csv("position.csv")

In [4]:
url_df = pd.DataFrame(url_pd)

In [5]:
result = []
result =url_df['0'].tolist()

In [6]:
title = []
type_ = []
date = []
location = []
experience = []
job_description = []
for url in result:
    response = get(url)
    # print(response)
    soup = bs(response.content,'lxml')
    a = soup.find("span", class_ = "jdp-title-job-title").text 
    b = soup.find("div", class_ = "job-date-posted").text[14:-1]
    c = soup.find("span", class_ = "jobLocation").text
    d = soup.find("div", class_ = "secondary-text-color").text
    e = soup.find("li", class_ = "job-experience")
    f = e.find("div", class_="secondary-text-color").text
    g = soup.find("div", class_="jdp-job-description-card content-card").text
    title.append(a)
    type_.append(d)
    date.append(b)
    location.append(c)
    experience.append(f)
    job_description.append(g)
len(title)

300

In [7]:
import pandas as pd
test_df = pd.DataFrame({"Url":result,
                        "Job Title":title,
                        "Post Date":date,
                        "Job Type": type_,
                        "Location":location,
                        "Experience":experience, 
                        "Job_Desciption":job_description})
test_df.to_csv('result.csv', sep=',', header=True, index=True)